# **Estruturas criptográficas: TP1 problema 2**

## Descrição do problema

No problema 2, utilizando o "package" **Cryptography**, foi-nos pedida uma implementação de:

1. Uma AEAD (**"authenticated encryption with associated data"**) com **“Tweakable Block Ciphers”**. A cifra por blocos primitiva, usada para gerar a “tweakable block cipher”, é o AES-256 ou o ChaCha20.
    
2. Um canal privado de informação assíncrona com acordo de chaves feito com **“X448 key exchange”** e **“Ed448 Signing&Verification”** para autenticação  dos agentes. Com uma fase de confirmação da chave acordada.

## Implementação

Em primerio lugar, começamos por implementar a **“Tweakable Block Ciphers”**, e para isso, optamos pela cifra por blocos primitiva **ChaCha20**.
De modo a cifrar a mensagem em blocos, dividimos a mesma em blocos de 


In [1]:
import sys
import struct, os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric.x448 import X448PrivateKey, X448PublicKey
from cryptography.hazmat.primitives.asymmetric.ed448 import Ed448PrivateKey, Ed448PublicKey
from cryptography.exceptions import InvalidSignature 
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
import math
import asyncio

tam_bloco = 16

        
async def handle_emitter_ex2(reader, writer):
    
    #### X448 Key Exchange
    
    private_key = X448PrivateKey.generate()
    public_key = private_key.public_key()
    
    
    # Envia as publicKeys
    writer.write(public_key.public_bytes_raw())
    await writer.drain()
    

    public_key_emitter = await reader.readexactly(56)
    public_key_emitter = X448PublicKey.from_public_bytes(public_key_emitter)
    
    verification_key = await reader.readexactly(57)
    verification_key = Ed448PublicKey.from_public_bytes(verification_key)
    
    shared_key = private_key.exchange(public_key_emitter)
    # Perform key derivation.
    key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=None,
    ).derive(shared_key)
    
    
    
    # Lê a tag
    signature = await reader.read(114)
    
    
    #Lê os dados associados
    ad = await reader.readuntil(b'_ADEnd_')
    iv = ad[:16]
    
    associated_data = ad[16:].decode()[:-7]
        
    num_blocos = int(associated_data.split('+')[1])
    padding = int(associated_data.split('+')[2])
    size = num_blocos * tam_bloco
    
    #Lê o criptograma
    ciphertext = await reader.readexactly(size)
        
    try:
        verification_key.verify(signature, ad + ciphertext)
    except InvalidSignature:
        print("Assinatura inválida.")
        return
    

    decrypted_text = b""

    tweak = iv
    for i in range(num_blocos):
    # Decifragem

        if i != num_blocos - 1:
            cipher = Cipher(algorithms.ChaCha20(key, tweak), mode=None, backend=default_backend())
            decryptor = cipher.decryptor()
            decrypted_text += decryptor.update(ciphertext[:tam_bloco]) + decryptor.finalize()

            ciphertext = ciphertext[tam_bloco:]
        else:
            cipher = Cipher(algorithms.ChaCha20(key, tweak), mode=None, backend=default_backend())
            decryptor = cipher.decryptor()
            decrypted_text += decryptor.update(ciphertext[:tam_bloco - padding]) + decryptor.finalize()

        tweak = incrementar_tweak(tweak)


    print(decrypted_text.decode())

    # writer.close()
        
#Emitter
async def send_message_ex2(reader, writer):
    
    #establecer concexão
    # reader, writer = await asyncio.open_connection('127.0.0.1', 8888)
    
    #### ED448 Signing&Verification

    ed_private_key = Ed448PrivateKey.generate()
    verification_key = ed_private_key.public_key()
    
    
    # Lê a publicKey
    public_key_receiver = await reader.readexactly(56)
    public_key_receiver = X448PublicKey.from_public_bytes(public_key_receiver)


    # Gera a chave privada e a pública
    private_key = X448PrivateKey.generate()
    public_key = private_key.public_key()
       
    # Envia a publicKey gerada
    writer.write(public_key.public_bytes_raw() + verification_key.public_bytes_raw())
    await writer.drain()

    # Obtem a chave compartilhada
    shared_key = private_key.exchange(public_key_receiver)
    # Perform key derivation.
    key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=None,
    ).derive(shared_key)
    

    message = input("Enter message: ").encode()
    
    blocos, pad = divide_em_blocos_com_padding(message, tam_bloco)    

    iv = os.urandom(16)

    ciphertext = b""

    tweak = iv
    for bloco in blocos:

        # Inicialização do objeto de cifra
        cipher = Cipher(algorithms.ChaCha20(key, tweak), mode=None, backend=default_backend())
        # Cifragem
        encryptor = cipher.encryptor()
        ciphertext += encryptor.update(bloco) + encryptor.finalize()

        tweak = incrementar_tweak(tweak)

    associated_data = iv + b'+' + str(len(blocos)).encode() + b'+' + str(pad).encode()

    ciphertext = associated_data + b'_ADEnd_' + ciphertext

    siganture = ed_private_key.sign(ciphertext)

    ciphertext = siganture + ciphertext
    

    # Envia a mensagem para o servidor
    writer.write(ciphertext)
    await writer.drain()

    # Fecha a conexão
    writer.close()
    await writer.wait_closed()
    

def divide_em_blocos_com_padding(mensagem, tamanho_bloco):
    bytes_adicionais = 0
    numero_blocos = math.ceil(len(mensagem) / tamanho_bloco)
    blocos = []
    for i in range(numero_blocos):
        inicio = i * tamanho_bloco
        fim = (i + 1) * tamanho_bloco
        bloco = mensagem[inicio: fim]
        if len(bloco) < tamanho_bloco:  # Adicionar padding com zeros
            bytes_adicionais = tamanho_bloco - len(bloco)
            bloco += b'\x00' * bytes_adicionais
        blocos.append(bloco)
    return blocos, bytes_adicionais

def incrementar_tweak(tweak):
    # Decodificar o tweak como um número inteiro de 128 bits (16 bytes)
    valor = int.from_bytes(tweak, byteorder='big')
    
    # Incrementar o valor do tweak
    novo_valor = valor + 1
    
    # Codificar o novo valor de volta para um tweak de 16 bytes
    novo_tweak = novo_valor.to_bytes(16, byteorder='big')
    
    return novo_tweak

#receiver
async def receiver_ex2():
    server = await asyncio.start_server(lambda reader, writer: handle_emitter_ex2(reader, writer), '127.0.0.1', 8888)

    print("Receiver ready...\n")

    async with server:
        await server.serve_forever()
        
#emitter
async def emitter_ex2():
        
    reader, writer = await asyncio.open_connection('127.0.0.1', 8888)
    return reader, writer
 

In [2]:
asyncio.create_task(receiver_ex2())

<Task pending name='Task-5' coro=<receiver_ex2() running at /tmp/ipykernel_41727/118439091.py:197>>

Receiver ready...



In [7]:
reader, writer = await emitter_ex2()

# Envia a mensagem
await send_message_ex2(reader, writer)


b'2E\x083-\xe7>\xc3\x17\t\xce\xaeaJ]~+1+2_ADEnd_'
['', '1', '2']
kvpervmwlev,we
